In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)) ## Check the shape
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1)) ## Check the shape
test_images = test_images.astype("float32") / 255
print(len(train_images))
print(len(train_labels))

In [ ]:
train_images_1 = []
train_labels_1 = []
train_images_2 = []
train_labels_2 = []


digit_counts = {digit: 0 for digit in range(5, 10)}

for i, digit in enumerate(train_labels):
    if digit <= 4:
        train_images_1.append(train_images[i])
        train_labels_1.append(digit)
    elif 5 <= digit <= 9 and digit_counts[digit] < 10:
        train_images_2.append(train_images[i])
        train_labels_2.append(digit)
        digit_counts[digit] += 1


print(len(train_images_1))
print(len(train_labels_1)) #Ensuring same length, and totaling about 30k.

print(len(train_images_2))
print(len(train_labels_2)) #Ensuring 50 count (10 of each digit > 4)

In [ ]:
test_images_1 = []
test_labels_1 = []
test_images_2 = []
test_labels_2 = []

for i, digit in enumerate(test_labels):
    if digit <= 4:
      test_images_1.append(test_images[i])
      test_labels_1.append(digit)
    else:
      test_images_2.append(test_images[i])
      test_labels_2.append(digit)

print(len(test_images_1))
print(len(test_labels_1))

print(len(test_images_2))
print(len(test_labels_2)) #Ensuring same length, total 5k each.

In [ ]:
#Shuffling training set 1 and 2:
import numpy as np

perm_1 = np.random.permutation(len(train_images_1))
train_images_1 = np.array(train_images_1)
train_labels_1 = np.array(train_labels_1) #Need to be numpy arrays

perm_2 = np.random.permutation(len(train_images_2))
train_images_2 = np.array(train_images_2)
train_labels_2 = np.array(train_labels_2) #Need to be numpy arrays

train_images_1 = train_images_1[perm_1]
train_labels_1 = train_labels_1[perm_1]

train_images_2 = train_images_2[perm_2]
train_labels_2 = train_labels_2[perm_2]



#Making test_data a numpy array so they can have shape
test_images_1 = np.array(test_images_1)
test_images_2 = np.array(test_images_2)
test_labels_1 = np.array(test_labels_1)
test_labels_2 = np.array(test_labels_2)

print(len(train_images_1))
print(len(train_images_2))

In [ ]:
#Creating validation data (20% of train)
val_images_1 = train_images_1[:int(.2*len(train_images_1))] #Takes first 20%
train_images_1 = train_images_1[int(.2*len(train_images_1)):] #Assigns training images last 80%

val_labels_1 = train_labels_1[:int(.2*len(train_labels_1))] #Takes first 20%
train_labels_1 = train_labels_1[int(.2*len(train_labels_1)):] #Assigns training labels last 80%

val_images_2 = train_images_2[:int(.2*len(train_images_2))] #Repeat for 2
train_images_2 = train_images_2[int(.2*len(train_images_2)):]

val_labels_2 = train_labels_2[:int(.2*len(train_labels_2))]
train_labels_2 = train_labels_2[int(.2*len(train_labels_2)):]

In [ ]:
#Model 1
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_1 = keras.Model(inputs=inputs, outputs=outputs)

model_1.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_1.fit(train_images_1, train_labels_1,
epochs=30,
validation_data=(val_images_1,val_labels_1),
callbacks=callbacks)

In [ ]:
#Print training/validation results for Model 1 at optimal epochs determined by ModelCheckpoint
test_model_1 = keras.models.load_model("Model_1.keras")
train_loss, train_acc = test_model_1.evaluate(train_images_1,train_labels_1)
test_loss, test_acc = test_model_1.evaluate(test_images_1,test_labels_1)
print("Model 1:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
#Model 2
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_2 = keras.Model(inputs=inputs, outputs=outputs)

model_2.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_2.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_2.fit(train_images_2, train_labels_2,
epochs=30,
validation_data=(val_images_2,val_labels_2),
callbacks=callbacks)

In [ ]:
#Print training/validation results for Model 2 at optimal epochs determined by ModelCheckpoint
test_model_2 = keras.models.load_model("Model_2.keras")
train_loss, train_acc = test_model_2.evaluate(train_images_2,train_labels_2)
test_loss, test_acc = test_model_2.evaluate(test_images_2,test_labels_2)
print("Model 2:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
#Data Augmentation Approach
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

augmented_images = []
augmented_labels = []

for i in range(len(train_images_2)):
    for _ in range(10): #Creates 10 new pictures per original
        augmented_image = data_augmentation(
            np.expand_dims(train_images_2[i], axis=0)
        )
        augmented_images.append(augmented_image[0])
        augmented_labels.append(train_labels_2[i])

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

train_images_2_new = np.concatenate((train_images_2, augmented_images))
train_labels_2_new = np.concatenate((train_labels_2, augmented_labels))

#Shuffling
perm = np.random.permutation(len(train_images_2_new))
train_images_2_new = train_images_2_new[perm]
train_labels_2_new = train_labels_2_new[perm]


#Creating new val and train
val_images_2_new = train_images_2_new[:int(.2*len(train_images_2_new))]
train_images_2_new = train_images_2_new[int(.2*len(train_images_2_new)):]

val_labels_2_new = train_labels_2_new[:int(.2*len(train_labels_2_new))]
train_labels_2_new = train_labels_2_new[int(.2*len(train_labels_2_new)):]

In [ ]:
#Model 3 (with augmentation)
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_3 = keras.Model(inputs=inputs, outputs=outputs)

model_3.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_3.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_3.fit(train_images_2_new, train_labels_2_new,
epochs=30,
validation_data=(val_images_2_new,val_labels_2_new),
callbacks=callbacks)

In [ ]:
#Print training/validation results for Model 3 at optimal epochs determined by ModelCheckpoint
test_model_3 = keras.models.load_model("Model_3.keras")
train_loss, train_acc = test_model_3.evaluate(train_images_2_new,train_labels_2_new)
test_loss, test_acc = test_model_3.evaluate(test_images_2,test_labels_2)
print("Model 3:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
print(len(train_images_2_new)) #(40 original + 400 augmented) * .8 = 352

In [ ]:
#New data augmentation: Model 4
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),  #Flips both horizontally and vertically
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),  #Shift up to 10% of the height/width
        layers.RandomRotation(0.2),  #Testing Rotation
        layers.RandomContrast(0.2),  #Testing Contrast
    ]
)

augmented_images = []
augmented_labels = []

for i in range(len(train_images_2)):
    for _ in range(20): #Trying 20 pictures per
        augmented_image = data_augmentation(
            np.expand_dims(train_images_2[i], axis=0)
        )
        augmented_images.append(augmented_image[0])
        augmented_labels.append(train_labels_2[i])

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

train_images_2_new = np.concatenate((train_images_2, augmented_images))
train_labels_2_new = np.concatenate((train_labels_2, augmented_labels))

#Shuffling
perm = np.random.permutation(len(train_images_2_new))
train_images_2_new = train_images_2_new[perm]
train_labels_2_new = train_labels_2_new[perm]


#Creating new val and train
val_images_2_new = train_images_2_new[:int(.2*len(train_images_2_new))]
train_images_2_new = train_images_2_new[int(.2*len(train_images_2_new)):]

val_labels_2_new = train_labels_2_new[:int(.2*len(train_labels_2_new))]
train_labels_2_new = train_labels_2_new[int(.2*len(train_labels_2_new)):]

In [ ]:
#Model 4 (with augmentation)
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_4 = keras.Model(inputs=inputs, outputs=outputs)

model_4.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_4.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_4.fit(train_images_2_new, train_labels_2_new,
epochs=30,
validation_data=(val_images_2_new,val_labels_2_new),
callbacks=callbacks)

In [ ]:
#Print training/validation results for Model 4 at optimal epochs determined by ModelCheckpoint
test_model_4 = keras.models.load_model("Model_4.keras")
train_loss, train_acc = test_model_4.evaluate(train_images_2_new,train_labels_2_new)
test_loss, test_acc = test_model_4.evaluate(test_images_2,test_labels_2)
print("Model 4:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
print(len(train_images_2_new)) #(40 original + 800 augmented) * .8 = 672

In [ ]:
#New data augmentation: Model 5
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.15),
        layers.RandomZoom(height_factor=(0.1, 0.3)),
        layers.RandomBrightness(factor=0.2),  #Testing brightness
    ]
)

augmented_images = []
augmented_labels = []

for i in range(len(train_images_2)):
    for _ in range(30): #Trying 20 pictures per
        augmented_image = data_augmentation(
            np.expand_dims(train_images_2[i], axis=0)
        )
        augmented_images.append(augmented_image[0])
        augmented_labels.append(train_labels_2[i])

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

train_images_2_new = np.concatenate((train_images_2, augmented_images))
train_labels_2_new = np.concatenate((train_labels_2, augmented_labels))

#Shuffling
perm = np.random.permutation(len(train_images_2_new))
train_images_2_new = train_images_2_new[perm]
train_labels_2_new = train_labels_2_new[perm]


#Creating new val and train
val_images_2_new = train_images_2_new[:int(.2*len(train_images_2_new))]
train_images_2_new = train_images_2_new[int(.2*len(train_images_2_new)):]

val_labels_2_new = train_labels_2_new[:int(.2*len(train_labels_2_new))]
train_labels_2_new = train_labels_2_new[int(.2*len(train_labels_2_new)):]

In [ ]:
#Model 5 (with augmentation)
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_5 = keras.Model(inputs=inputs, outputs=outputs)

model_5.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_5.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_5.fit(train_images_2_new, train_labels_2_new,
epochs=30,
validation_data=(val_images_2_new,val_labels_2_new),
callbacks=callbacks)

In [ ]:
#Print training/validation results for Model 4 at optimal epochs determined by ModelCheckpoint
test_model_5 = keras.models.load_model("Model_5.keras")
train_loss, train_acc = test_model_5.evaluate(train_images_2_new,train_labels_2_new)
test_loss, test_acc = test_model_5.evaluate(test_images_2,test_labels_2)
print("Model 5:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
print(len(train_images_2_new)) #(40 original + 1200 augmented) * .8 = 992